Clustering simulated sequences using TMRCA

Name: CQS21 
FYP 2024

In [ ]:
# installing packages to my python environment
# NOTE: my pip is v22.0.4 but I cant seem to update to the newest 24.0

#!pip install zarr
#!pip install scipy
#!pip install scikit-allel
#!pip install matplotlib
#!pip install tqdm
#!pip install dask
#!pip install seaborn

In [2]:
# Import modules
import numpy as np
import zarr
#import allel   
#idk why but my allel doesnt seem to load?? even though ive definitely installed it as scikit-allel
import scipy.cluster.hierarchy as sch
import scipy.spatial
import matplotlib
import matplotlib.pyplot as plt
import scipy.signal
from scipy.ndimage import gaussian_filter1d
from numpy.lib.stride_tricks import sliding_window_view
from tqdm import tqdm
import dask
from dask import compute, delayed
from itertools import combinations
import time
import seaborn as sns
import sys

Step 1. Generate burn in simulations establishing nucleotide diversity across a range of parameters (DONE BY PREVIOUS STUDENTS)

In [ ]:
## use HPC
1. SHELL SCRIPT for HPC BELOW #run test for just 1 burn in
#---------------------------------------------------------------------------------------------------------
!/bin/bash
PBS -N run_pairwise    ###REMOVE ME -  Change this to set the name of your script (can i set a script in SLIM?)
PBS -j oe
PBS -k oe             

PBS -m ae

PBS -l walltime=48:00:00
PBS -l select=1:ncpus=4:mem=5gb          #job parameters update after trial
                                  
module load anaconda3                     #loading modules: what modules do I need? is there a module for slim?
source activate f2                       # what does this f2 mean?


## to direct output to cwd, use $PBS_O_WORKDIR:
## specify LOGFILE found in ~/ during execution then moved to cwd on job completion
##
cd $PBS_O_WORKDIR
JOBNUM=`echo $PBS_JOBID | sed 's/\..*//'`
LOGFILE=${PBS_JOBNAME}.o${JOBNUM}

#########################################
##                                     ##
## Output some useful job information. ##
##                                     ##
#########################################

echo ------------------------------------------------------
echo -n 'Job is running on node '; cat $PBS_NODEFILE
echo ------------------------------------------------------
echo PBS: qsub is running on $PBS_O_HOST
echo PBS: originating queue is $PBS_O_QUEUE
echo PBS: executing queue is $PBS_QUEUE
echo PBS: working directory is $PBS_O_WORKDIR
echo PBS: execution mode is $PBS_ENVIRONMENT
echo PBS: job identifier is $PBS_JOBID
echo PBS: job name is $PBS_JOBNAME
echo PBS: job number is $JOBNUM
echo PBS: logfile is $LOGFILE
echo PBS: node file is $PBS_NODEFILE
echo PBS: current home directory is $PBS_O_HOME
#echo PBS: PATH = $PBS_O_PATH
echo ------------------------------------------------------


# move LOGFILE to cwd
##
mv $HOME/$LOGFILE $PBS_O_WORKDIR



#--------------------------------------------------------------------------------------------------------
2. submit shell script using resource manager
qsub slim_simulations.py

In [ ]:
##HPC example run script from Josh Reynolds
#!/bin/bash

#PBS -N run_pairwise    ###REMOVE ME -  Change this to set the name of your script
#PBS -j oe
#PBS -k oe

#PBS -m ae

#PBS -l walltime=72:00:00	###REMOVE ME - change this to set your time (hh:mm:ss)
#PBS -l select=1:ncpus=4:mem=20gb       ###REMOVE ME - leave select at 1, choose your cpus and memory

## NB values for ncpus and mem are allocated
## to each node (specified by select=N)
##
## to direct output to cwd, use $PBS_O_WORKDIR:
## specify LOGFILE found in ~/ during execution then moved to cwd on job completion
##
cd $PBS_O_WORKDIR
JOBNUM=`echo $PBS_JOBID | sed 's/\..*//'`
LOGFILE=${PBS_JOBNAME}.o${JOBNUM}

#########################################
##                                     ##
## Output some useful job information. ##
##                                     ##
#########################################

echo ------------------------------------------------------
echo -n 'Job is running on node '; cat $PBS_NODEFILE
echo ------------------------------------------------------
echo PBS: qsub is running on $PBS_O_HOST
echo PBS: originating queue is $PBS_O_QUEUE
echo PBS: executing queue is $PBS_QUEUE
echo PBS: working directory is $PBS_O_WORKDIR
echo PBS: execution mode is $PBS_ENVIRONMENT
echo PBS: job identifier is $PBS_JOBID
echo PBS: job name is $PBS_JOBNAME
echo PBS: job number is $JOBNUM
echo PBS: logfile is $LOGFILE
echo PBS: node file is $PBS_NODEFILE
echo PBS: current home directory is $PBS_O_HOME
#echo PBS: PATH = $PBS_O_PATH
echo ------------------------------------------------------

## load common modules as standard
##
module load anaconda3/personal vcftools   ### load any modules you need here (e.g. vcftools, ibdseq). To see a list of available modules, use "module avail" on your login node
source activate f2

## command timed to get mem and wallclock info
##

R --vanilla --quiet --slave < ~/f2/scripts/pairwise_dif.R   ### Then supply your command. Either write the code here, or in a separate shell script that you then call on here.

## move LOGFILE to cwd
##
mv $HOME/$LOGFILE $PBS_O_WORKDIR


In [ ]:
## SLIM code file burnin_no.1_read.slim

initialize() {


        // Use tree-sequence recording to speed up burn-in
        initializeTreeSeq();

        defineConstant('pop_size',100);
        //defineConstant('T_N',10*pop_size);
        defineConstant('ge_length',70000);
        defineConstant('sweep_site',integerDiv((ge_length+1),2));
        defineConstant('low_site',sweep_site-1);
        defineConstant('high_site',1+sweep_site);
        

        defineConstant('recombination_rate',0.00025);
        defineConstant('mut_rate',0.00025);
        //defineConstant('T_mu', 10/mut_rate); //round it
        defineConstant('nucleotide_diversity', (4*pop_size*mut_rate)/(1+(2*4*pop_size*mut_rate)));
        initializeRecombinationRate(recombination_rate);
        initializeMutationRate(mut_rate);

        // m1 is a neutral mutation
        initializeMutationType('m0', 0.5, 'f', 0.0);
        m0.mutationStackPolicy = "l";   

        initializeGenomicElementType('g1', m0, 1.0 );
        initializeGenomicElement(g1, 0, low_site);
        initializeGenomicElement(g1, sweep_site, sweep_site);
        initializeGenomicElement(g1, high_site, ge_length);}


1 early(){
	sim.addSubpop("p0", pop_size);
}
//this command was later added to various files on the high-performance computer.
1:1000 late(){
	div = calcHeterozygosity(p0.genomes);
	catn(sim.cycle +","+ div);
}
       
//conditional termination of somulation based on val of nucleotide diversity
500:39999 late() {
div = calcHeterozygosity(p0.genomes);
if(div >= nucleotide_diversity*1.1){ // if(div >= nucleotide_diversity){
sim.simulationFinished();
catn('number of generations taken ' + sim.cycle);
sim.treeSeqOutput("burnin_no.1_early_10.trees");
}
}


// Run burn-in for ~10/mu generations
40000 late() {
        // Output recorded tree-sequence
        sim.treeSeqOutput("burnin_no.1._10_.trees");
}


Step 2. Conduct 27 variations of partial soft sweep simulations and output vcf files using the HPC facility. Complete x number of repeats.

Size of population, N: 100, 1000, 10000

Mutation rate, μ : (0.1/4N), (1/4N), (5/4N)

Recombination rate, r : (0.1/4N), (1/4N), (5/4N)

In [6]:
import numpy as np

# Define parameter values
N_values = [100, 1000, 10000]
mutation_rates = [0.1 / (4 * N) for N in N_values]
recombination_rates = [0.1 / (4 * N) for N in N_values]

# Generate combinations of parameters
parameter_combinations = [(N, mu, r) for N in N_values for mu in mutation_rates for r in recombination_rates]

# Ensure we have exactly 27 combinations
assert len(parameter_combinations) == 27, "Number of combinations must be 27"

# Assign file numbers to combinations
burn_in_files = list(range(1, 28))  # List of file numbers from 1 to 27
combinations_matrix = np.zeros(27, dtype=[('N', int), ('mu', float), ('r', float), ('file', int)])
for index, (N, mu, r) in enumerate(parameter_combinations):
    combinations_matrix[index] = (N, mu, r, burn_in_files[index])

# Save the matrix to a text file
np.savetxt("parameter_combinations.txt", combinations_matrix, fmt="%d,%.6f,%.6f,%d", delimiter=",",
           header="N, mu, r, file", comments="")



In [ ]:
#DO NOT RUN 
1. SHELL SCRIPT for HPC BELOW

!/bin/bash
PBS -l walltime=24:00:00
PBS -l select=1:ncpus=30:mem=60gb          #job parameters update after trial
PBS -J 1-10                                  #array job for 27 combinations?
module load    ????                      #loading modules: what modules do I need? is there a module for slim?
cd $PBS_O_WORKDIR
slim_simulations.py                                 # my script in python


2. submit shell script using resource manager
qsub slim_simulations.py



##PBS -J n-m

#in your jobscript, where n and m are integers, e.g 1-4 if you wanted four runs. 
#You can then supply these to your slim command using the variable $PBS_ARRAY_INDEX with this taking on the value of J. 
#You'll then need something in your slim code to interpret and convert these values.



# choosing a wall time 
# Sensible values are 8, 24 hours, 48 hours or 72 hours. There is little benefit in selecting intermediate values.
#If possible, use checkpointing, i.e. save intermediate results that can be used to restart your program.



In [ ]:
# python script for matrix iteration? with array run

#The code uses $PBS_ARRAY_INDEX to execute the correct iteration of the loop that processes just one element of the matrix.
#All array runs combined process all elements of the matrix in parallel.


#slide 97 of the Intro to HPC wiki pdf
input = np.loadtxt("parameter_combinations.txt", dtype='f', delimiter=',')
counter = 0
array_index = int(os.environ['PBS_ARRAY_INDEX'])
for i in range(0,input.shape[0]):
    for j in range(0, input.shape[1]):
        counter = counter+1
        if counter == array_index:
            print('processing element', input[i,j])
            # your code goes in here
            
            # how to get it to run in SLIM???

            #SLIM Script below



Step 3. Read in VCF Files from soft sweep simulations, extract haplotypes using scikit_allel

Working with 27 simulation variants and repeats of each variant (100?)

but why only look at haplotypes of first 200 individuals? and why calculate sample frequency over specifically 400 haplotypes? is it linked to the 200 individuals = 400 genomes?


In [21]:
seed = sys.argv[1]
#prev student: Like Hamming distance code, this was also taken from Anushka Thawani. Adaptations were made to this on the 
#high-performance computer using shell script, but this could not be represented.
def convert(file, genome):
    '''
    This function extracts haplotypes sequences from a vcf file 
    Adapted from: http://alimanfoo.github.io/2018/04/09/selecting-variants.html 
    
    Arguments:
        file: name of vcf file (from SLiM soft sweep simulation)
        genome: length of genome used in SLiM simulation 
        
    Returns:
        ht: haplotype sequences for 200 individuals
        samp_freq: frequency of sweep mutation in sample
        cols: used to color dendrogram

    '''
    
    v = file + '.vcf'
    z = file + '.zarr'
    slim_sim_data = allel.read_vcf(v, fields='*')
    allel.vcf_to_zarr(v, z, fields='*', overwrite=True)
    data = zarr.open_group(z, mode='r')
    
 
    pos = allel.SortedIndex(data['variants/POS']) # Stores the ID and genomic position of each variant
    
    # Extract genotypes for the first 200 individuals and convert to haplotypes
    gt = data['calldata/GT'][:,0:200] 
    ht = allel.GenotypeArray(gt).to_haplotypes()
    
    mutation = int((genome+1)/2) + 1  # position of sweep mutation
    
    
    # Output the frequency of the sweep mutation in the sample
    contains_sweep = pos.locate_range(mutation,mutation)
    sweep = ht[contains_sweep]
    sweep = np.sum(sweep, axis =0)
    
    samp_freq = np.sum(sweep)/400  # 400 haplotypes
    
    
    # This dictionary is used later to color the dendrogram branches according to whether or not the 
    # corresponding sequence contains the sweep mutation
    cols = {}
    for i in range(400):
        if sweep[i]:
            cols[i] = 'r'  
        else:
            cols[i] = "#808080"
    
    return ht, pos, samp_freq, cols, sweep

Step 4. Calculate Hij (S or homozygosity) for all pairs of haplotypes
make a function 
homozygosity = (no. of SNPs/ length of window, L)

In [3]:
def sliding_homozygosity (length, genome, ht, sweeploc, pos):
    '''
    This function calculates the sliding homozygosity for all pairs of haplotypes.
    
    Arguments:
    length: length of sliding window
    genome : length of genome 
    ht : vector?  of haplotype sequences from convert() function in previous codeblock
    pos: position of variants from convert() function in previous codeblock
    sweeploc: position of sweep mutation in genome

    
    Returns:
    homozygosities:  homozygosity of all haplotypes in ht in a array(?)
    '''
    # Make empty vectors
    homozygosities = []
    
    #iterate through all nucleotides in the genome
    for x in range(0,genome):
        #define nt ranges 
        start  = x
        end = x + length
        # locate the position of region around a variant Nt
        region = pos.locate_range(start,end) 
        haplotype_region = ht [region]

        #use allel.pairwise distance
        pairwise_dist = allel.pairwise_distance(haplotype_region, metric = 'hamming' , chunked=True, blen=None) #can i chunk = true to speed up computation? 
        homozygosities = pairwise_dist/length

    return homozygosities



## TO DO streamlining: previous students code say to add a if condition (region =/= prev region to speed up code)

Step 5. Calculate Lij (shared haplotype length) from Hij by finding width at half maximum homozygosity

In [ ]:
#half_max_homozygosity = (max(homozygosities) + min(homozygosities))/2
threshold = 0.87  #setting threshold to calculate trough points in each haplotype

#Defining function to find troughs, to be used in calculating Lij in another function
def finding_troughs(smooth, sweeploc):
    '''
    This function finds troughs for a pair of haplotype sequences
    
    Arguments:
        sliding homozygosity: smoothed sliding window homozygosity for all pairs of sequences
        sweeploc: position of sweep mutation in genome (same as previous function)
        
    Returns:
        lower: position of breakpoint left of the sweep site
        upper: position of breakpoint right of the sweep site
        SHL: shared haplotype length
    '''
    
    #finding troughs
    troughs = scipy.signal.find_peaks(-smooth) #- sign inverts the graph so the 'peaks' are our troughs
    troughs = troughs[0]     # Indexes of all troughs
    troughs = troughs[smooth[troughs] < threshold]   # Extract troughs where homozygosity<threshold
    
    #finds the peaks
    peaks = scipy.signal.find_peaks(smooth)
    peaks = peaks[0]  #indexes peaks
    
    # Find positions of troughs flanking sweep site
    bp = np.searchsorted(troughs,sweeploc)  #search sorted finds index of position where mutation should be inserted in order to maintain the same order
    lower = troughs[bp - 1] #index of sweep site -1
    upper = troughs[bp]  #index of sweep site
    
    # Find the average peak position around the sweep site
    highest = peaks[(peaks >= lower) & (peaks <= upper)]
    if highest.size != 0:
        highest = np.mean(highest)
    else: 
        highest = (lower+upper)/2
    
    
    lower = (lower+highest)/2
    upper = (upper+highest)/2

    SHL = upper - lower
    
    return int(lower), int(upper), SHL



In [ ]:
## function using each haplotype pair to return SHL and find lower and upper limits of SHL
# uses finding_troughs()
def find_breakpoint(haplotype_pair):
    '''
    For a pair of sequences, this function smoothes the sliding homozygosity and returns the SHL
    Arguments:
        haplotype_pair: a pair of haplotype sequences
        
    Returns:
        lower: position of breakpoint left of the sweep site
        upper: position of breakpoint right of the sweep site
        SHL: shared haplotype length
    '''
    
    mutation_pos = mutation 
    smooth = gaussian_filter1d(haplotype_pair, points_g)
    try:
        lower, upper, SHL = finding_troughs(smooth, mutation_pos)
    except IndexError:
        lower = -1.3
        upper = -1.3
        SHL = -1.3
        
    return lower, upper, SHL

Step 6. Calculate Tij (Time to common ancestor) from Lij and Kij (which is no. of SNPs) on each shared length (haplotype)

𝜏_𝑖𝑗=(𝑘_𝑖𝑗+1)/(2ℓ_𝑖𝑗 (𝑟+𝜇))

In [ ]:
# Calculating Kij (No. of SNPs in each SHL)

def calculating_kij(n,gts,ht,result_find_breakpoint,pos):
    '''
    This function finds the number of SNPs over the shared haplotype length for all pairs of haplotype sequences
    
    Arguments:
        n: number of haplotype sequences
        gts: number of haplotype pairs
        ht: haplotype sequnces
        result_find_breakpoint: output from find_breakpoint function
        
    Returns:
        diffs: number of SNPs for all pairs of haplotype sequences
        
    '''
    pairwise = []
    for combo in combinations(list(range(0,n)), 2): 
        pairwise.append(combo)

    diffs = np.empty(shape=(gts),dtype=np.float32)
    for i in range(gts):
        pair = ht[:,pairwise[i]]
        try:
            start = result_find_breakpoint[i,1]
            stop = result_find_breakpoint[i,2]

            window_pos = pos.locate_range(start, stop)
            window = pair[window_pos]

            d = allel.pairwise_distance(window, metric = "hamming")

            diffs[i]=d 

        except KeyError:
            diffs[i]=-1.3 
    
    return diffs

In [ ]:
#Calculating Tij, Time to Common Ancestor (TMRCA) using mutation rate and number of SNPs

In [ ]:
## PREV STUDENTS SCRIPT FOR REFERENCE
# delete after doing my own


def analysis(file,genome,pop,window,threshold,points,r=1,u=1): 
    '''
    This function clusters the sequences stored in a .vcf file.
    
    Arguments:
        file: name of vcf file
        genome: length of genome (in SLiM simulation)
        pop: effective population size (in SLiM simulation)
        window: length of sliding window
        threshold: threshold above which troughs are ignored
        points: number of points to use for 1D-gaussian filter (see scipy documentation)
        r: recombination rate
        u: mutation rate 
    '''
    
    print(file)

    global mutation
    mutation = int((genome+1)/2) 
    global thresh
    thresh=threshold
    global points_g
    points_g = points
    
    # Extract haplotype sequences from .vcf file
    ht, pos, samp_freq, cols, sweep = convert(file, genome)

    
    # Calculate sliding homozygosity for all pairs of haplotype sequences
    L=window
    n = 400 #number of haplotypes 
    gts = int((n*(n-1))/2)
    hom = sliding_distance(L, mutation, genome, ht, pos, gts)

    
    # Find SHL for all pairs of haplotype sequences 
    hom_dask = dask.array.from_array(hom, chunks=(genome,1)) 
    hom = []
    results = dask.array.apply_along_axis(find_breakpoint,0,hom_dask) 
    results_computed = results.compute()

    # Manipulating the dataframe to make it easier to process
    results_computed = np.transpose(results_computed)
    index = np.asarray(range(0,gts))
    index = np.expand_dims(index, axis=0)
    results_computed_1 = np.concatenate((index.T, results_computed), axis=1)
    
    
    # Calculate the TMRCA from the SHLs and number of SNPs
    r = r/(2*pop)
    u = u/(100*pop)
    shls = results_computed_1[:,3]   # SHLs for all pairs of haplotype sequences 
    shls[shls<=0] = genome
    diffs = find_snp(n,gts,ht,results_computed_1,pos)  # SNPs for all pairs of haplotype sequences 
    snp = (1+(diffs*shls))/(2*shls*(r + u)) # TMRCA metric for all pairs of haplotype sequences 

    
    # Remove negative and non-integer TMRCA values
    impute = np.nanmean(snp)
    x = np.isfinite(snp)
    for i in np.where(x == 0)[0]:
        snp[i] = impute
    snp[snp<=0] = impute  
    
    
    # Clustering 
    Z = sch.linkage(snp, method = 'complete')
    
    
    # Plot dendrogram without colouring branches
    matplotlib.rcParams.update({'font.size': 24})
    fig = plt.figure(figsize=(30, 12))
    gs = matplotlib.gridspec.GridSpec(2, 1, hspace=0.1, wspace=1, height_ratios=(1,1))

    ax_dend = fig.add_subplot(gs[0, 0])
    sns.despine(ax=ax_dend, offset=5, bottom=True, top=True)
    dd = sch.dendrogram(Z,color_threshold=0,above_threshold_color='#808080',ax=ax_dend)

    ls = []
    for leaf, leaf_color in zip(plt.gca().get_xticklabels(), dd["leaves_color_list"]):
        leaf.set_color(cols[int(leaf.get_text())])
        ls.append(int(leaf.get_text()))

    ax_dend.set_ylabel('Haplotype age/generations',fontsize=24)
    ax_dend.set_title('Haplotype clusters',fontsize=24)
    
    
    # Plot dendrogram and colour branches
    
    ax_dend_2 = fig.add_subplot(gs[1, 0])
    
    dflt_col = "#808080"
    
    link_cols = {}
    for i, i12 in enumerate(Z[:,:2].astype(int)):
        c1, c2 = (link_cols[x] if x > len(Z) else cols[x] for x in i12)
        link_cols[i+1+len(Z)] = c1 if c1 == c2 else dflt_col

    sns.despine(ax=ax_dend_2, offset=5, bottom=True, top=True)
    dd = sch.dendrogram(Z,color_threshold=None,link_color_func=lambda x: link_cols[x],ax=ax_dend_2)

    ls = []
    for leaf, leaf_color in zip(plt.gca().get_xticklabels(), dd["leaves_color_list"]):
        leaf.set_color(cols[int(leaf.get_text())])
        ls.append(int(leaf.get_text()))

    ax_dend_2.set_ylabel('Haplotype age/generations',fontsize=24)
    
    
    # Save dendrogram
    output = 'accurate_' + file + '.pdf'
    plt.savefig(output)  
        
    return 